In [2]:
import tensorflow as tf

cnn_graph_path = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\model\ckp-10000.meta'
cnn_model_path = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\model\ckp-10000'
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(cnn_graph_path)
    saver.restore(sess, cnn_model_path)
    tensor_inputs2 = tf.get_default_graph().get_tensor_by_name('Placeholder:0')
    tensor_outpus2 = tf.get_default_graph().get_tensor_by_name('metrics/ArgMax:0')
    tensor_keep_prob2 = tf.get_default_graph().get_tensor_by_name('keep_prob:0')
    conv_node = tf.get_default_graph().get_tensor_by_name('cnn/conv1d/conv1d/Conv2D:0')
    print(conv_node)
    print(tensor_inputs2)

INFO:tensorflow:Restoring parameters from .\deep_learn\jd_deep_learn\jd_lstm_runout\model\ckp-10000
Tensor("cnn/conv1d/conv1d/Conv2D:0", shape=(100, 1, 118, 256), dtype=float32)
Tensor("Placeholder:0", shape=(100, 120), dtype=int32)


In [2]:
from tensorflow.python.framework import graph_util
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(cnn_graph_path)
    saver.restore(sess, cnn_model_path)
    
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph_def,['metrics/ArgMax']
    )
    with tf.gfile.GFile('.\deep_learn\jd_deep_learn\jd_lstm_runout\model\jd_comment_cnn_model.pb', 'w') as gf:
        serialized_graph = output_graph_def.SerializeToString()
        gf.write(serialized_graph)


INFO:tensorflow:Restoring parameters from .\deep_learn\jd_deep_learn\jd_lstm_runout\model\ckp-10000
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 7 variables.
INFO:tensorflow:Converted 7 variables to const ops.


In [9]:
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
import os
export = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\model\jd_comment_cnn_serving'
graph_pb = '.\deep_learn\jd_deep_learn\jd_lstm_runout\model\jd_comment_cnn_model.pb'
# if not os.path.exists(export):
#     os.mkdir(export)
# if not tf.gfile.Exists(export):
#     tf.gfile.MakeDirs(export)
builder = tf.saved_model.builder.SavedModelBuilder(export)

if tf.gfile.Exists(export):
    tf.gfile.MakeDirs(export)
with tf.gfile.GFile(graph_pb, 'rb') as f:
    graph_def_ = tf.GraphDef()
    graph_def_.ParseFromString(f.read())

sigs = {}
with tf.Session(graph=tf.Graph()) as sess:
    tf.import_graph_def(graph_def_, name='')
    g = tf.get_default_graph()
    inp = g.get_tensor_by_name('Placeholder:0')
    oup = g.get_tensor_by_name('metrics/ArgMax:0')
    keep = g.get_tensor_by_name('keep_prob:0')
    
    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.saved_model.signature_def_utils.predict_signature_def(
        {'in': inp}, {'out': oup}
    )
    builder.add_meta_graph_and_variables(sess, [tag_constants.SERVING],
                                         signature_def_map=sigs)
builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: .\deep_learn\jd_deep_learn\jd_lstm_runout\model\jd_comment_cnn_serving\saved_model.pb


b'.\\deep_learn\\jd_deep_learn\\jd_lstm_runout\\model\\jd_comment_cnn_serving\\saved_model.pb'